# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    words = set(text)
    words=list(words)
    vocab_to_int = dict()
    int_to_vocab=dict()
    for i in range(len(words)):
        vocab_to_int[words[i]]=i
        int_to_vocab[i]=words[i]
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punctuation_tokenizer=dict()
    punctuation_tokenizer["."]="||Period||"
    punctuation_tokenizer[","]="||Comma||"
    punctuation_tokenizer["\""]="||QuotationMark||"
    punctuation_tokenizer[";"]="||Semicolon||"
    punctuation_tokenizer["!"]="||ExclamationMark||"
    punctuation_tokenizer["?"]="||QuestionMark||"
    punctuation_tokenizer["("]="||LeftParenthesis||"
    punctuation_tokenizer[")"]="||RightParenthesis||"
    punctuation_tokenizer["--"]="||Dash||"
    punctuation_tokenizer["\n"]="||Return||"
    return punctuation_tokenizer

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    inputs = (tf.placeholder(tf.float32, shape=[None,None], name="input"), \
              tf.placeholder(tf.float32, shape=[None,None], name="targets"), \
             tf.placeholder(tf.float32, name="LearningRate")\
             )
    return inputs


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    rnn_layer_size = 2
    cell = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(rnn_size) for _ in range(rnn_layer_size)])
    init_state = tf.identity(cell.zero_state(batch_size, tf.float32), name="initial_state")
    return cell, init_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    word_embeddings = tf.get_variable("word_embeddings", [vocab_size, embed_dim])
    embedded_input = tf.nn.embedding_lookup(word_embeddings, input_data)
    return embedded_input


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype =tf.float32)
    final_state = tf.identity(final_state, name = "final_state")
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    print("rnn size", rnn_size, "input data", input_data, "vocab_size", vocab_size, "embed dim", embed_dim)
    embedded_inputs = get_embed(input_data, vocab_size, embed_dim)
    logits, final_state = build_rnn(cell, embedded_inputs)
    logits = tf.layers.dense(logits, vocab_size, activation=None)
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

rnn size 256 input data Tensor("Placeholder:0", shape=(128, 5), dtype=int32) vocab_size 27 embed dim 300
Tests Passed


###### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [23]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    num_batches = int(len(int_text)/(batch_size * seq_length))
    print("text int length", len(int_text))
    print("batch size", batch_size)
    print("seq_length", seq_length)
    print("num batches ", num_batches)
    batches = np.zeros((num_batches, 2, batch_size, seq_length), dtype="int")
    print("batches shape ", batches.shape)
    for batch_no in range(0,num_batches):
        for i in range(0,batch_size):
            for j in range (0,seq_length):
                position_formula = batch_no*seq_length + j + i*2*seq_length
                print("batch no ", batch_no, "i ", i, "j ", j, "position formula ", position_formula)
                batches[batch_no,0,i,j] = int_text[position_formula]
                batches[batch_no,1,i,j] = int_text[position_formula+1]
    print (batches)
    return batches


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

text int length 5000
batch size 128
seq_length 5
num batches  7
batches shape  (7, 2, 128, 5)
batch no  0 i  0 j  0 position formula  0
batch no  0 i  0 j  1 position formula  1
batch no  0 i  0 j  2 position formula  2
batch no  0 i  0 j  3 position formula  3
batch no  0 i  0 j  4 position formula  4
batch no  0 i  1 j  0 position formula  10
batch no  0 i  1 j  1 position formula  11
batch no  0 i  1 j  2 position formula  12
batch no  0 i  1 j  3 position formula  13
batch no  0 i  1 j  4 position formula  14
batch no  0 i  2 j  0 position formula  20
batch no  0 i  2 j  1 position formula  21
batch no  0 i  2 j  2 position formula  22
batch no  0 i  2 j  3 position formula  23
batch no  0 i  2 j  4 position formula  24
batch no  0 i  3 j  0 position formula  30
batch no  0 i  3 j  1 position formula  31
batch no  0 i  3 j  2 position formula  32
batch no  0 i  3 j  3 position formula  33
batch no  0 i  3 j  4 position formula  34
batch no  0 i  4 j  0 position formula  40
batch no

batch no  0 i  56 j  1 position formula  561
batch no  0 i  56 j  2 position formula  562
batch no  0 i  56 j  3 position formula  563
batch no  0 i  56 j  4 position formula  564
batch no  0 i  57 j  0 position formula  570
batch no  0 i  57 j  1 position formula  571
batch no  0 i  57 j  2 position formula  572
batch no  0 i  57 j  3 position formula  573
batch no  0 i  57 j  4 position formula  574
batch no  0 i  58 j  0 position formula  580
batch no  0 i  58 j  1 position formula  581
batch no  0 i  58 j  2 position formula  582
batch no  0 i  58 j  3 position formula  583
batch no  0 i  58 j  4 position formula  584
batch no  0 i  59 j  0 position formula  590
batch no  0 i  59 j  1 position formula  591
batch no  0 i  59 j  2 position formula  592
batch no  0 i  59 j  3 position formula  593
batch no  0 i  59 j  4 position formula  594
batch no  0 i  60 j  0 position formula  600
batch no  0 i  60 j  1 position formula  601
batch no  0 i  60 j  2 position formula  602
batch no  

batch no  0 i  106 j  1 position formula  1061
batch no  0 i  106 j  2 position formula  1062
batch no  0 i  106 j  3 position formula  1063
batch no  0 i  106 j  4 position formula  1064
batch no  0 i  107 j  0 position formula  1070
batch no  0 i  107 j  1 position formula  1071
batch no  0 i  107 j  2 position formula  1072
batch no  0 i  107 j  3 position formula  1073
batch no  0 i  107 j  4 position formula  1074
batch no  0 i  108 j  0 position formula  1080
batch no  0 i  108 j  1 position formula  1081
batch no  0 i  108 j  2 position formula  1082
batch no  0 i  108 j  3 position formula  1083
batch no  0 i  108 j  4 position formula  1084
batch no  0 i  109 j  0 position formula  1090
batch no  0 i  109 j  1 position formula  1091
batch no  0 i  109 j  2 position formula  1092
batch no  0 i  109 j  3 position formula  1093
batch no  0 i  109 j  4 position formula  1094
batch no  0 i  110 j  0 position formula  1100
batch no  0 i  110 j  1 position formula  1101
batch no  0 i

batch no  1 i  28 j  1 position formula  286
batch no  1 i  28 j  2 position formula  287
batch no  1 i  28 j  3 position formula  288
batch no  1 i  28 j  4 position formula  289
batch no  1 i  29 j  0 position formula  295
batch no  1 i  29 j  1 position formula  296
batch no  1 i  29 j  2 position formula  297
batch no  1 i  29 j  3 position formula  298
batch no  1 i  29 j  4 position formula  299
batch no  1 i  30 j  0 position formula  305
batch no  1 i  30 j  1 position formula  306
batch no  1 i  30 j  2 position formula  307
batch no  1 i  30 j  3 position formula  308
batch no  1 i  30 j  4 position formula  309
batch no  1 i  31 j  0 position formula  315
batch no  1 i  31 j  1 position formula  316
batch no  1 i  31 j  2 position formula  317
batch no  1 i  31 j  3 position formula  318
batch no  1 i  31 j  4 position formula  319
batch no  1 i  32 j  0 position formula  325
batch no  1 i  32 j  1 position formula  326
batch no  1 i  32 j  2 position formula  327
batch no  

batch no  1 i  78 j  1 position formula  786
batch no  1 i  78 j  2 position formula  787
batch no  1 i  78 j  3 position formula  788
batch no  1 i  78 j  4 position formula  789
batch no  1 i  79 j  0 position formula  795
batch no  1 i  79 j  1 position formula  796
batch no  1 i  79 j  2 position formula  797
batch no  1 i  79 j  3 position formula  798
batch no  1 i  79 j  4 position formula  799
batch no  1 i  80 j  0 position formula  805
batch no  1 i  80 j  1 position formula  806
batch no  1 i  80 j  2 position formula  807
batch no  1 i  80 j  3 position formula  808
batch no  1 i  80 j  4 position formula  809
batch no  1 i  81 j  0 position formula  815
batch no  1 i  81 j  1 position formula  816
batch no  1 i  81 j  2 position formula  817
batch no  1 i  81 j  3 position formula  818
batch no  1 i  81 j  4 position formula  819
batch no  1 i  82 j  0 position formula  825
batch no  1 i  82 j  1 position formula  826
batch no  1 i  82 j  2 position formula  827
batch no  

...

batch no  5 i  103 j  3 position formula  1058
batch no  5 i  103 j  4 position formula  1059
batch no  5 i  104 j  0 position formula  1065
batch no  5 i  104 j  1 position formula  1066
batch no  5 i  104 j  2 position formula  1067
batch no  5 i  104 j  3 position formula  1068
batch no  5 i  104 j  4 position formula  1069
batch no  5 i  105 j  0 position formula  1075
batch no  5 i  105 j  1 position formula  1076
batch no  5 i  105 j  2 position formula  1077
batch no  5 i  105 j  3 position formula  1078
batch no  5 i  105 j  4 position formula  1079
batch no  5 i  106 j  0 position formula  1085
batch no  5 i  106 j  1 position formula  1086
batch no  5 i  106 j  2 position formula  1087
batch no  5 i  106 j  3 position formula  1088
batch no  5 i  106 j  4 position formula  1089
batch no  5 i  107 j  0 position formula  1095
batch no  5 i  107 j  1 position formula  1096
batch no  5 i  107 j  2 position formula  1097
batch no  5 i  107 j  3 position formula  1098
batch no  5 i

batch no  6 i  25 j  3 position formula  283
batch no  6 i  25 j  4 position formula  284
batch no  6 i  26 j  0 position formula  290
batch no  6 i  26 j  1 position formula  291
batch no  6 i  26 j  2 position formula  292
batch no  6 i  26 j  3 position formula  293
batch no  6 i  26 j  4 position formula  294
batch no  6 i  27 j  0 position formula  300
batch no  6 i  27 j  1 position formula  301
batch no  6 i  27 j  2 position formula  302
batch no  6 i  27 j  3 position formula  303
batch no  6 i  27 j  4 position formula  304
batch no  6 i  28 j  0 position formula  310
batch no  6 i  28 j  1 position formula  311
batch no  6 i  28 j  2 position formula  312
batch no  6 i  28 j  3 position formula  313
batch no  6 i  28 j  4 position formula  314
batch no  6 i  29 j  0 position formula  320
batch no  6 i  29 j  1 position formula  321
batch no  6 i  29 j  2 position formula  322
batch no  6 i  29 j  3 position formula  323
batch no  6 i  29 j  4 position formula  324
batch no  

batch no  6 i  75 j  3 position formula  783
batch no  6 i  75 j  4 position formula  784
batch no  6 i  76 j  0 position formula  790
batch no  6 i  76 j  1 position formula  791
batch no  6 i  76 j  2 position formula  792
batch no  6 i  76 j  3 position formula  793
batch no  6 i  76 j  4 position formula  794
batch no  6 i  77 j  0 position formula  800
batch no  6 i  77 j  1 position formula  801
batch no  6 i  77 j  2 position formula  802
batch no  6 i  77 j  3 position formula  803
batch no  6 i  77 j  4 position formula  804
batch no  6 i  78 j  0 position formula  810
batch no  6 i  78 j  1 position formula  811
batch no  6 i  78 j  2 position formula  812
batch no  6 i  78 j  3 position formula  813
batch no  6 i  78 j  4 position formula  814
batch no  6 i  79 j  0 position formula  820
batch no  6 i  79 j  1 position formula  821
batch no  6 i  79 j  2 position formula  822
batch no  6 i  79 j  3 position formula  823
batch no  6 i  79 j  4 position formula  824
batch no  

batch no  6 i  125 j  2 position formula  1282
batch no  6 i  125 j  3 position formula  1283
batch no  6 i  125 j  4 position formula  1284
batch no  6 i  126 j  0 position formula  1290
batch no  6 i  126 j  1 position formula  1291
batch no  6 i  126 j  2 position formula  1292
batch no  6 i  126 j  3 position formula  1293
batch no  6 i  126 j  4 position formula  1294
batch no  6 i  127 j  0 position formula  1300
batch no  6 i  127 j  1 position formula  1301
batch no  6 i  127 j  2 position formula  1302
batch no  6 i  127 j  3 position formula  1303
batch no  6 i  127 j  4 position formula  1304
[[[[   0    1    2    3    4]
   [  10   11   12   13   14]
   [  20   21   22   23   24]
   ...
   [1250 1251 1252 1253 1254]
   [1260 1261 1262 1263 1264]
   [1270 1271 1272 1273 1274]]

  [[   1    2    3    4    5]
   [  11   12   13   14   15]
   [  21   22   23   24   25]
   ...
   [1251 1252 1253 1254 1255]
   [1261 1262 1263 1264 1265]
   [1271 1272 1273 1274 1275]]]


 [[[   5 

AssertionError: Batches returned wrong contents. For example, input sequence 1 in the first batch was [10 11 12 13 14]

## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [ ]:
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 500
# Sequence Length
seq_length = 10
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 1000

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [ ]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    return None, None, None, None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [ ]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    return None


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [ ]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.